# Data Collection

**We use twitter API key for data fetching and dataset generation**

In [1]:
#!pip install tweepy
import tweepy
import csv
import pandas as pd
from tqdm import tqdm
import numpy as np
import ast
import string
import datetime

# input your credentials here
consumer_key = 'EcwAltcOolBlwQlesRg2Ux5A8'
consumer_secret = 'tMltHbelqqDAWJhoTSPa2TDrB4LWL4FK9B40F7qGsU7wUEmN9J'
access_token = '1331612828281683973-9SYgUjN2pYpoTVpONPRh3SCsMsMbB1'
access_token_secret = 'n901ArAC0NHH1EExcCEn8BMApnGfjejHM6V0Cz3wtS5Mq'

#accessing data using twitter api via tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

###### **Fetching and saving tweets that contains words like 'sad' 'disappoint' 'hurtful' 'upset'**

In [2]:
#a csv file for storing depressive tweets 
csvFile = open('tweet-dataset-depressive.csv', 'a')

# Use csv Writer
csvWriter = csv.writer(csvFile)
query = [ 'sad' , 'disappoint' , 'hurtful' , 'upset']

csvWriter.writerow(["id", "timestamp","tweet_text"])
for qu in query:
    print(qu)
    for tweet in tqdm(tweepy.Cursor(api.search, q=qu,tweet_mode="extended",lang="en",since="2020-04-01").items(5000)):
        if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
            csvWriter.writerow([tweet.id, tweet.created_at,tweet.full_text.encode('utf-8')])
        
    print("Processing for the word {} finished".format(qu))

###### **Fetching and saving tweets that contains words like 'happy' 'satisfied'**

In [3]:
#csv file for storing normal tweets
csvFile = open('tweet-dataset-non-depressive.csv', 'a')

# Use csv Writer
csvWriter = csv.writer(csvFile)
# 'happy', 'blissful', 'satisfied', 'delighted' , 'pleased' ,
query = ['joyful', 'cheerful', 'merry',  'glad']

csvWriter.writerow(["id", "timestamp","tweet_text"])
for qu in query:
    print(qu)
    for tweet in tqdm(tweepy.Cursor(api.search, q=qu,tweet_mode="extended",\
                                    lang="en",since="2020-04-01").items(6000)):
        if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
            csvWriter.writerow([tweet.id, tweet.created_at,tweet.full_text.encode('utf-8')])
        
    print("Processing for {} finished".format(qu))

#### **working on Datasets**

In [4]:
# reading csv file in df_depressive
df_depressive = pd.read_csv("Depressive_Merged.csv",encoding="utf-8")

In [5]:
#df_depressive.columns = ["id","time_stamp","tweet_text"]
print(len(df_depressive))
df_depressive.head()

7696


,id,timestamp,tweet_text
0,1.388080e+18,30-04-2021 10:31,"b""am i mentally ill or is wearing this hair ti..."
1,1.388080e+18,30-04-2021 10:31,b'Bath Salts are used to treat mental and phys...
2,1.388080e+18,30-04-2021 10:30,b'Insomnia and Anxiety really pulled up and be...
3,1.388080e+18,30-04-2021 10:30,b'@NNinchop hab leider n social anxiety :('
4,1.388080e+18,30-04-2021 10:30,b'I go to the tip yesterday after my jolly to ...


In [6]:
# reading csv file containing non-depressive words in tweets
df_normal = pd.read_csv("Happy_Merged.csv",encoding="utf-8")

In [7]:
#df_normal.columns = ["id","time_stamp","tweet_text"]
print(len(df_normal))
df_normal.head()

9296


,id,timestamp,tweet_text
0,1.390000e+18,30-04-2021 21:41,"b'@madamjujujive Yes, just fancied it. It\xe2\..."
1,1.390000e+18,30-04-2021 21:40,b'Great Relief and Blissful. As Bobde out its ...
2,1.390000e+18,30-04-2021 21:36,b'@Haarish_Shah My parents did register to sta...
3,1.390000e+18,30-04-2021 21:36,"b""Writing a ship I genuinely love has inverted..."
4,1.390000e+18,30-04-2021 21:35,b'@kumar_isu this might help you https://t.co/...


In [8]:
# if want to drop some rows
#df_normal = df_normal.drop([0,1]) -> drops 1st and 2nd rows.
df_normal.head()

,id,timestamp,tweet_text
0,1.390000e+18,30-04-2021 21:41,"b'@madamjujujive Yes, just fancied it. It\xe2\..."
1,1.390000e+18,30-04-2021 21:40,b'Great Relief and Blissful. As Bobde out its ...
2,1.390000e+18,30-04-2021 21:36,b'@Haarish_Shah My parents did register to sta...
3,1.390000e+18,30-04-2021 21:36,"b""Writing a ship I genuinely love has inverted..."
4,1.390000e+18,30-04-2021 21:35,b'@kumar_isu this might help you https://t.co/...


#### **Defining functions for data cleaning**

In [9]:
# !pip install emoji
import re
import emoji
import ast

# function for replacing urls(starting from http) with an empty string.
def remove_urls(text):
    return re.sub(r'http\S+', '', text)

# funtion for replacing usernames(starting with @___) with an empty string
def remove_usernames(text):
    return re.sub('@[^\s]+','',text)

def remove_hashtags(text):
    return re.sub('[!@_^%*#$=-]', '', text)

def remove_emojis(text):
    allchars = [j for j in text]
    emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([k for k in text.split() if not any(i in k for i in emoji_list)])
    return clean_text

def remove_digits(text):
    return re.sub(r'\b\d+\b', '', text)

def remove_digits_str(text):
    return re.sub(r'\d+', '', text)



In [10]:
# Data Cleaning on Dataset containing Depressive words
df_depressive['tweet_text'] = df_depressive['tweet_text'].apply(ast.literal_eval).str.decode("utf-8")

# .apply() is an inbuilt function in pandas
df_depressive.tweet_text = df_depressive.tweet_text.apply(remove_urls)
df_depressive['tweet_text'].dropna(inplace=True) # drops null string
df_depressive.tweet_text = df_depressive.tweet_text.apply(remove_usernames)
df_depressive.tweet_text = df_depressive.tweet_text.apply(lambda x:''.join([i for i in x if i not in string.punctuation]))
df_depressive.tweet_text = df_depressive.tweet_text.apply(remove_emojis)
df_depressive.tweet_text = df_depressive.tweet_text.apply(remove_hashtags)
df_depressive.tweet_text = df_depressive.tweet_text.apply(remove_digits)
df_depressive.tweet_text = df_depressive.tweet_text.apply(remove_digits_str)

df_depressive['tweet_text'].dropna(inplace=True) # considering only tweet_text column
df_depressive = df_depressive.drop_duplicates(keep='first') # dropping duplicates (keeping 1st)... when same timestamp , id , tweet 
# considering whole table for matching id , time and text 
df_depressive.head()

,id,timestamp,tweet_text
0,1.388080e+18,30-04-2021 10:31,am i mentally ill or is wearing this hair tie ...
1,1.388080e+18,30-04-2021 10:31,Bath Salts are used to treat mental and physic...
2,1.388080e+18,30-04-2021 10:30,Insomnia and Anxiety really pulled up and beat...
3,1.388080e+18,30-04-2021 10:30,hab leider n social anxiety
4,1.388080e+18,30-04-2021 10:30,I go to the tip yesterday after my jolly to IK...


In [11]:
# checking length of dataset after data cleaning

print(len(df_depressive))
df_depressive['tweet_text'][1]

7465


'Bath Salts are used to treat mental and physical health ailments It is believed to provide several health benefits for people with muscle pain and stiffness stiff aching joints arthritis circulation problems headache anxiety amp stress skin allergies dry and itchy skin'

In [12]:
# remove emojis
df_depressive = df_depressive.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

In [13]:
# saving dataframe to new .csv file

df_depressive.to_csv("1-output-depressive.csv",index=False)

In [14]:
print(len(df_depressive))

7465


In [15]:
df_normal.head()

,id,timestamp,tweet_text
0,1.390000e+18,30-04-2021 21:41,"b'@madamjujujive Yes, just fancied it. It\xe2\..."
1,1.390000e+18,30-04-2021 21:40,b'Great Relief and Blissful. As Bobde out its ...
2,1.390000e+18,30-04-2021 21:36,b'@Haarish_Shah My parents did register to sta...
3,1.390000e+18,30-04-2021 21:36,"b""Writing a ship I genuinely love has inverted..."
4,1.390000e+18,30-04-2021 21:35,b'@kumar_isu this might help you https://t.co/...


In [16]:
# Data Cleaning on Dataset containing Non-Depressive words
df_normal['tweet_text'] = df_normal['tweet_text'].apply(ast.literal_eval).str.decode("utf-8")
df_normal.tweet_text = df_normal.tweet_text.apply(remove_urls)
df_normal['tweet_text'].dropna(inplace=True) # drops null string
df_normal.tweet_text = df_normal.tweet_text.apply(remove_usernames)
df_normal.tweet_text = df_normal.tweet_text.apply(lambda x:''.join([i for i in x if i not in string.punctuation]))

df_normal.tweet_text = df_normal.tweet_text.apply(remove_emojis)
df_normal.tweet_text = df_normal.tweet_text.apply(remove_hashtags)
df_normal.tweet_text = df_normal.tweet_text.apply(remove_digits)
df_normal.tweet_text = df_normal.tweet_text.apply(remove_digits_str)


df_normal['tweet_text'].dropna(inplace=True)
df_normal = df_normal.drop_duplicates(keep='first')
df_normal.head()

,id,timestamp,tweet_text
0,1.390000e+18,30-04-2021 21:41,Yes just fancied it It’s blissful
1,1.390000e+18,30-04-2021 21:40,Great Relief and Blissful As Bobde out its Gar...
2,1.390000e+18,30-04-2021 21:36,My parents did register to stand a better chan...
3,1.390000e+18,30-04-2021 21:36,Writing a ship I genuinely love has inverted m...
4,1.390000e+18,30-04-2021 21:35,this might help you


In [17]:
df_normal['tweet_text'][7051]

'I think she actually took that in the ER waiting room The last place I would expect to see a pleased expression to be sure after letting your child suffer for  days'

In [18]:
# remove emojis 
df_normal = df_normal.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

In [19]:
print(len(df_normal))

9057


In [20]:
df_normal.to_csv("1-output-normal.csv",index=False)